# Day 9

## Part 1

We need to edit our Intcode interpreter to support a new parameter mode called "relative".

> Parameters in mode 2, relative mode, behave very similarly to parameters in position mode: the parameter is interpreted as a position. Like position mode, parameters in relative mode can be read from or written to.
>
> The important difference is that relative mode parameters don't count from address 0. Instead, they count from a value called the relative base. The relative base starts at 0.
>
> The address a relative mode parameter refers to is itself plus the current relative base. When the relative base is 0, relative mode parameters and position mode parameters with the same value refer to the same address.

In [19]:
from collections import defaultdict


class Computer:
    def __init__(self, program, read_input, write_output):
        self.position = 0
        self.relative_base = 0
        self.parameters_mode = defaultdict(lambda: 0)
        self.program = program.copy()
        self.read_input = read_input
        self.write_output = write_output
        
    @staticmethod
    def log(*args):
        if False:
            print(*args)

    def get_param_value(self, param):
        param_address = self.get_param_address(param)
        return self.program[param_address]

    def get_param_address(self, param):
        mode = self.parameters_mode[param]

        if mode == 0:  # position mode
            return self.program[self.position + param]
        elif mode == 1:  # immediate mode
            return self.position + param
        elif mode == 2:  # relative mode:
            param_position = self.position + param
            return self.relative_base + self.program[param_position]
        else:
            raise ValueError("Unknown paremeter mode")

    def read_instruction(self):
        # read instruction and split it into opcode and parameters mode
        try:
            instruction = self.program[self.position]
        except IndexError:
            # end of program
            return None

        opcode = instruction % 100

        value = instruction // 100
        i = 1
        while value > 0:
            self.parameters_mode[i] = value % 10
            value //= 10
            i += 1

        self.log(self.position, '>', instruction, opcode, dict(self.parameters_mode))
        return opcode

    async def compute(self):
        """Compute the final state of a program, and return it.

        `program` is a list of int.
        `read_input` is a method returning a value each time it is called.
        `write_output` is a method taking a value as parameter.
        """
        # init state
        self.position = 0

        # run program
        while True:
            opcode = self.read_instruction()
            if opcode is None:
                # end of program
                log("Stop: EOF")
                return

            # execute opcode
            if opcode == 1:
                # add x y
                x = self.get_param_value(1)
                y = self.get_param_value(2)
                result_addr = self.get_param_address(3)

                result = x + y
                self.program[result_addr] = result
                self.log("add", x, y, result_addr, result)

                step = 4
            elif opcode == 2:
                # mult x y
                x = self.get_param_value(1)
                y = self.get_param_value(2)
                result_addr = self.get_param_address(3)

                result = x * y
                self.program[result_addr] = result
                self.log("mult", x, y, result_addr, result)

                step = 4
            elif opcode == 3:
                # read x
                result_addr = self.get_param_address(1)

                value = await self.read_input()
                self.program[result_addr] = value
                self.log("read", value, result_addr)

                step = 2
            elif opcode == 4:
                # write x
                value = self.get_param_value(1)
                await self.write_output(value)
                self.log("write", value)

                step = 2
            elif opcode == 5:
                # jump-if-true x y
                value = self.get_param_value(1)
                jump_addr = self.get_param_value(2)

                self.log("jump-if-true", value, jump_addr)

                if value:
                    self.position = jump_addr
                    step = 0
                else:
                    step = 3
            elif opcode == 6:
                # jump-if-false x y
                value = self.get_param_value(1)
                jump_addr = self.get_param_value(2)

                self.log("jump-if-false", value, jump_addr)

                if not value:
                    self.position = jump_addr
                    step = 0
                else:
                    step = 3
            elif opcode == 7:
                # lt x y z
                x = self.get_param_value(1)
                y = self.get_param_value(2)
                result_addr = self.get_param_address(3)

                slef.log("lt", x, y, result_addr)

                if x < y:
                    self.program[result_addr] = 1
                else:
                    self.program[result_addr] = 0

                step = 4
            elif opcode == 8:
                # eq x y z
                x = self.get_param_value(1)
                y = self.get_param_value(2)
                result_addr = self.get_param_address(3)

                self.log("eq", x, y, result_addr)

                if x == y:
                    self.program[result_addr] = 1
                else:
                    self.program[result_addr] = 0

                step = 4
            elif opcode == 99:
                # end of program
                self.log("stop")
                break
            else:
                # unknown instruction
                self.log("unknown opcode")
                break

            self.position += step

        return self.program

### Tests

Read input and return the same value.

In [22]:
%autoawait trio

import trio

program = [3,0,4,0,99]

output, input_ = trio.open_memory_channel(1)
async def read_input():
    return await input_.receive()

async def write_output(value):
    await output.send(value)
    
DEBUG = True
await output.send(42)
await Computer(program, read_input, write_output).compute()
await input_.receive()

42